In [1]:
import pip 
import subprocess
subprocess.run(['pip', 'install', '-r', 'requirements.txt'])
import nest_asyncio
nest_asyncio.apply()
import asyncio
import vl_convert as vlc
import nltk
import pandas as pd
import csv
import re
import toolz
import io
import os
import altair_saver
import lyricsgenius
import requests
import altair as alt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
nltk.download('wordnet')
from lyricsgenius import Genius
import numpy as np
from dotenv import load_dotenv
import discord 
from discord.ext import commands
from requests.exceptions import HTTPError, Timeout

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached discord-2.3.2-py3-none-any.whl (1.1 kB)
  Using cached lyricsgenius-3.0.1-py3-none-any.whl (59 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached discord.py-2.3.2-py3-none-any.whl (1.1 MB)
  Using cached altair_saver-0.5.0-py3-none-any.whl (89 kB)
  Using cached vl_convert_python-1.2.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.7 MB)
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (773 kB)
  Using cached altair_data_server-0.4.1-py3-none-any.whl (12 kB)
  Using cached altair_viewer-0.4.0-py3-none-any.whl (844 kB)
  Using cached selenium-4.16.0-py3-none-any.whl (10.0 MB)
  Using cached portpicker-1.6.0-py3-none-any.whl (16 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached trio-0.23.2-py3-none-any.whl (461 kB)
  Using cached outcome-1.3.0.post0-

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
load_dotenv('3510.env')

True

In [3]:
Token = os.getenv('DISCORD_TOKEN')
intents = discord.Intents().all()

In [4]:
try:
    Genius(Token)
except HTTPError as e:
    print(e.errno)    # status code
    print(e.args[0])  # status code
    print(e.args[1])  # error message
except Timeout:
    pass

In [5]:
lemmatizer = WordNetLemmatizer()

api_key = None  

with open('API key.txt', 'r') as api_file:
    api_key = api_file.read().strip() 
genius = lyricsgenius.Genius(api_key)

def process_title(title, artist):
    if not title.strip() or not artist.strip():
        return None
    song = genius.search_song(title, artist)
    
    if song:
        lyrics = song.lyrics
      
        lemmatized_lyrics = ' '.join([lemmatizer.lemmatize(word) for word in re.sub(r"\n{1,2}", ". ", re.sub(r"\?\n{1,2}", "? ", lyrics)).split()])
        sentiment_polarity = TextBlob(lemmatized_lyrics).sentiment.polarity
        return sentiment_polarity
    else:
        return None


In [6]:
def process_subj(title, artist):
    if not title.strip() or not artist.strip():
        return None

    song = genius.search_song(title, artist)
    
    if song:
        lyrics = song.lyrics

        lemmatized_lyrics = ' '.join([lemmatizer.lemmatize(word) for word in re.sub(r"\n{1,2}", ". ", re.sub(r"\?\n{1,2}", "? ", lyrics)).split()])
        subjectivity = TextBlob(lemmatized_lyrics).sentiment.subjectivity
        return subjectivity
    else:
        return None


In [7]:
def fetch_lyrics(title, artist):
    if not title.strip() or not artist.strip():
        return None
    
    song = genius.search_song(title, artist)
    
    if song:
        lyrics = song.lyrics
        return lyrics

In [8]:
def lyrics_finder(lyrics):   
    request = genius.search_all(lyrics)
    for hit in request['sections'][2]['hits']:
        return hit['result']['title'] + ' by ' + hit['result']['primary_artist']['name']

In [9]:
def album_gets(album, artist):
    search_query = f'{album} {artist}'
    album_search = genius.search_albums(search_query, per_page=1)
    if 'sections' in album_search and album_search['sections']:
        first_section = album_search['sections'][0]
        hits = first_section.get('hits', [])
        if hits:
            first_hit = hits[0]
            result = first_hit.get('result', {})
            album_id = result.get('id')
            album_cover = result.get('cover_art_url')  
    album_tracks = genius.album_tracks(album_id, per_page=50)
    songs = []
    if 'tracks' in album_tracks:
        for track in album_tracks['tracks']:
            song_title = track.get('song', {}).get('title')
            songs.append(song_title)
    if songs == []:
        return [], None 
    return songs, album_cover


In [10]:
def visualizer(songs, artist):
    sentScores = []
    subjScores = []
    
    for song in songs:
        sentScores.append(process_title(song, artist))
        subjScores.append(process_subj(song, artist))
                          
    data = pd.DataFrame({
        'song_title': songs,
        'sentiment': sentScores,
        'subjectivity': subjScores})
    
    avg_sent_score = data['sentiment'].mean()
    avg_subj_score = data['subjectivity'].mean()
    
    visual = alt.Chart(data).mark_rect().encode(
        x=alt.X("song_title:N"),
        y=alt.Y("sentiment:Q", title="Sentiment Score"),
        color=alt.Color("subjectivity:Q"),
    ).properties(
        title="Album visualization of sentiment and subjectivity of songs"
    )
    
    return data, visual, avg_sent_score, avg_subj_score


In [11]:
bot = commands.Bot(command_prefix='!', intents = intents)

In [ ]:
@bot.event
async def on_ready():
    print('{0} has connected to Discord.'.format(bot.user.id, bot.user.name))
    for guild in bot.guilds:
        for channel in guild.text_channels:
            if channel.permissions_for(guild.me).send_messages:
                await channel.send("Bot has joined the server!")
                await channel.send('Please type !start to get started')
                break
@bot.command(name="start", help="Instructions for bot")
async def rogan(ctx):
    message = (
        'This discord bot has an array of commands please use freely\n'
        'If servers time out please rerun\n'
        'Inputs are case sensitive so make sure you enter your inputs properly\n'
        'Please do not use multiple commands at the same time\n'
        'Some Commands may take a few minutes to fully send to discord\n'
        'lyricGet command only works with puncuation and proper spelling, have not figured out how to make it not case sensitive\n'
        'The commands for this bot are:\n'
        '`!songSubj`: This gives a song subjectivity score\n'
        '`!songSent`: Gives a song sentiment score\n'
        '`!songLyrics`: Prints song lyrics\n'
        '`!lyricGet`: gets song based of inputed lyrics and artist\n'
        '`!albumGet`: gets songs from an inputed album\n'
        '`!albumGetProp`: gets songs from an inputed album but includes properties and visualization\n'
        '`!albumGetCover`: gets album cover from input\n'
        'When using commands !songsubj, !songsent, !songlyrics, write the song name, then add a comma, and then the artist name\n'
        'When using commands !lyricget ,write the lyrics, remember it is caseSensitve and use proper punctuation\n'
        'When using commands !albumGet, !albumGetProp, !albumGetCover, write the Album name, then add a comma, and then the artist name\n'
        'Example !albumGetProp Graduation, Kanye West\n'
        'Example !songLyrics Eleanor Rigby, Beatles\n'
    )
    await ctx.send(message)
           
@bot.command(name="albumGet", help="gets list of songs from an album")
async def songlook(ctx, *, song_info):
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return

    album, artist = map(str.strip, split_info)
    songs,album_cover = album_gets(album, artist)
    if not songs:
        await ctx.send("Album not found.")
        return
    await ctx.send('The Album ' + album + ' include these tracks by ' + artist)
    for song in songs:
        if song is not None:
            await ctx.send(f"{song}")
        else:
            await ctx.send(f"Song names for '{album}' by {artist} are unavailable.")
            
@bot.command(name="albumGetProp", help="gets list of songs from an album with their sentiment, subjectivity scores, album cover, and viusalization ")
async def songlook(ctx, *, song_info):
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return
    await ctx.send("Processing may take a few minutes")
    album, artist = map(str.strip, split_info)
    songs,album_cover = album_gets(album, artist)
    if not songs:
        await ctx.send("Album not found.")
        return
    data, visual ,avg_sent_score, avg_subj_score = visualizer(songs, artist)
    if not data.empty:
        formatted_data = data.to_string(index=True)  
        await ctx.send(f"Here's the data:\n```{formatted_data}```")
        await ctx.send(f"The average Sentiment Score of this album is: {round(avg_sent_score,2)}\n and the average subjectivity score of this album is: {round(avg_sent_score,2)}\n")
        await ctx.send(f"Album Cover:\n{album_cover}")
    else:
        await ctx.send(f"Album Properties for '{album}' by {artist} are unavailable.")
    
    if visual:
        png_data = vlc.vegalite_to_png(visual.to_json(), scale=2)
        with open("chart.png", "wb") as f:
            f.write(png_data)
        file_obj = discord.File("chart.png", filename="chart.png")
        await ctx.send("Album Visualization", file=file_obj)
     
@bot.command(name="albumGetCover", help="gets Album Cover for inputed album")
async def songlook(ctx, *, song_info):
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return
    album, artist = map(str.strip, split_info)
    songs,album_cover = album_gets(album, artist)
    if not songs:
        await ctx.send("Album not found.")
        return
    if songs:
        await ctx.send(f"Album Cover:\n{album_cover}")
    else:
        await ctx.send(f"Album cover for '{album}' by {artist} are unavailable.")
    
@bot.command(name="songSent", help="fetch song print sentiment")
async def songlook(ctx, *, song_info):

    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Album Name, Artist Name")
        return
    
    song_title, artist_name = map(str.strip, split_info)
    
    sent = process_title(song_title, artist_name)
    
    if sent is not None:
        await ctx.send(f"Sentiment polarity of '{song_title}' by {artist_name}: {round(sent,2)}")
    else:
        await ctx.send("Song not found or lyrics unavailable.")
     
@bot.command(name="songSubj", help="fetch song print subjectivity")
async def songsubj(ctx, *, song_info):
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Song Title, Artist Name")
        return
    
    song_title, artist_name = map(str.strip, split_info)
    
    subj = process_subj(song_title, artist_name)
    
    if subj is not None:
        await ctx.send(f"Subjectivity polarity of '{song_title}' by {artist_name}: {round(subj,2)}")
    else:
        await ctx.send("Song not found or lyrics unavailable.")


@bot.command(name="songLyrics", help="fetch song print lyrics")
async def songlyrics(ctx, *, song_info):
    split_info = song_info.split(',', 1)
    if len(split_info) != 2:
        await ctx.send("Please use the format: Song Title, Artist Name")
        return
    
    song_title, artist_name = map(str.strip, split_info)
    
    lyrics = fetch_lyrics(song_title, artist_name)
    
    if lyrics:
        chunks = lyrics.split('\n\n')  
        
        for chunk in chunks:
            await ctx.send(f"{song_title}, {artist_name}:\n{chunk}")
    else:
        await ctx.send("Song not found or lyrics unavailable.")
        
@bot.command(name="lyricGet", help="Searches for a song containing inputed lyrics")
async def getsong(ctx, *, song_info):
    get = lyrics_finder(song_info)
    print(f"Returned value from lyrics_finder(): {get}")  
    
    if get:  
        await ctx.send(f"Song name is '{get}'")
    else:
        await ctx.send("Song not found or lyrics unavailable.")
        
bot.run(Token)

[2023-12-21 01:39:45] [INFO    ] discord.client: logging in using static token
[2023-12-21 01:39:46] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 6dc86109a0019f674089e4a5a4cdf88d).


1172617540392992768 has connected to Discord.
